In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns

In [2]:
%cd /home/stepan/kaggle/cars_from_auction

/home/stepan/kaggle/cars_from_auction


In [3]:
%ls

data/  models/  prepare/  result/  review/  test/  train/


In [4]:
%ls data/

processed/  sample_submission.csv  test.csv  train.csv  zipcodes.csv


In [5]:
data_path = 'data/'
data_file = 'train.csv'

In [6]:
data = pd.read_csv(data_path + data_file, index_col=0)

In [7]:
data

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,price
48298,2.0,bus,2006,auto,140,c4,150000,gasoline,citroen,0.0,49191,380.0,4267
81047,NaN,NaN,2016,NaN,0,vito,150000,NaN,mercedes_benz,NaN,45896,NaN,2457
92754,2.2,limousine,2010,manual,175,mondeo,125000,diesel,ford,0.0,59229,930.0,10374
46007,NaN,NaN,2000,auto,265,andere,150000,gasoline,ford,0.0,39365,680.0,7098
76981,NaN,convertible,3,manual,109,2_reihe,150000,gasoline,peugeot,0.0,55271,NaN,2365
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50429,1.4,limousine,2006,manual,75,golf,90000,gasoline,volkswagen,0.0,35745,500.0,4686
64425,1.3,small car,4,manual,60,fiesta,150000,gasoline,ford,0.0,60386,NaN,864
90761,NaN,limousine,1996,manual,150,5er,150000,gasoline,bmw,0.0,28309,130.0,2275
39709,NaN,limousine,2007,manual,122,1er,100000,diesel,bmw,0.0,83623,500.0,8144


In [8]:
data.describe()

,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price,price
count,19950.000000,50000.000000,50000.000000,50000.000000,41734.000000,50000.000000,42669.000000,50000.000000
mean,1.867213,1839.195200,121.105060,125206.200000,0.096157,51436.403920,421.345239,5746.904380
std,0.808440,545.974243,188.787938,39587.836838,0.294809,25808.985658,679.444359,7688.683102
min,0.000000,0.000000,0.000000,5000.000000,0.000000,1067.000000,10.000000,455.000000
25%,1.400000,1998.000000,75.000000,100000.000000,0.000000,30989.000000,100.000000,1365.000000
50%,1.800000,2003.000000,110.000000,150000.000000,0.000000,50374.000000,230.000000,3185.000000
75%,2.000000,2008.000000,150.000000,150000.000000,0.000000,72415.000000,510.000000,7270.000000
max,9.500000,2016.000000,16311.000000,150000.000000,1.000000,99998.000000,38960.000000,163800.000000


# Transform

In [9]:
def transform_year(d):
    d = d.copy()
    year = d['registration_year']
    
    d.loc[(year < 1900) & (year <= 20), 'registration_year'] += 2000
    d.loc[(year < 1900) & (year > 20), 'registration_year'] += 1900
    return d

In [10]:
def transform_power(d):
    d = d.copy()
    power = d['power']
    car_type = d['type']
    
    d.loc[power >= 500, 'power'] = power.mean() / 2
    d.loc[(power >= 500) & (car_type == 'bus'), 'power'] = power.mean()
    d.loc[(power >= 500) & (car_type == 'small car'), 'power'] = power.mean() / 3
    
    d.loc[power < 25, 'power'] = power.mean() / 2
    d.loc[(power < 25) & (car_type == 'bus'), 'power'] = power.mean()
    d.loc[(power < 25) & (car_type == 'small car'), 'power'] = power.mean() / 3
    return d

In [11]:
def transform_capacity(d):
    d = d.copy()
    engine = d['engine_capacity']
    d.loc[engine > 3.3, 'engine_capacity'] = engine.mean()
    d.loc[engine < 0.8, 'engine_capacity'] = engine.mean()
    return d

In [12]:
def transform_price(d):
    d = d.copy()
    d.loc[:, 'insurance_price'] = np.log(d['insurance_price'])
    if 'price' in d:
        d.loc[:, 'price'] = np.log(d['price'])
    return d

## Apply

In [13]:
transformers = [transform_year, transform_power, transform_capacity, transform_price]

In [14]:
new_data = data
for t in transformers:
    new_data = t(new_data)

In [15]:
new_data.describe()

,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price,price
count,19950.000000,50000.000000,50000.000000,50000.000000,41734.000000,50000.000000,42669.000000,50000.000000
mean,1.783256,2003.777200,121.868216,125206.200000,0.096157,51436.403920,5.471285,8.084422
std,0.470074,7.269804,60.148077,39587.836838,0.294809,25808.985658,1.052609,1.068681
min,0.800000,1945.000000,25.000000,5000.000000,0.000000,1067.000000,2.302585,6.120297
25%,1.400000,2000.000000,75.000000,100000.000000,0.000000,30989.000000,4.605170,7.218910
50%,1.800000,2004.000000,110.000000,150000.000000,0.000000,50374.000000,5.438079,8.066208
75%,2.000000,2008.000000,150.000000,150000.000000,0.000000,72415.000000,6.234411,8.891512
max,3.300000,2016.000000,489.000000,150000.000000,1.000000,99998.000000,10.570291,12.006401


In [16]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 48298 to 25524
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   engine_capacity    19950 non-null  float64
 1   type               43746 non-null  object 
 2   registration_year  50000 non-null  int64  
 3   gearbox            47959 non-null  object 
 4   power              50000 non-null  float64
 5   model              47743 non-null  object 
 6   mileage            50000 non-null  int64  
 7   fuel               46417 non-null  object 
 8   brand              50000 non-null  object 
 9   damage             41734 non-null  float64
 10  zipcode            50000 non-null  int64  
 11  insurance_price    42669 non-null  float64
 12  price              50000 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 5.3+ MB


In [17]:
new_data.to_csv(data_path + 'processed/transformed_' + data_file)